<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist @ Mail.Ru Group <br>All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #10 (demo)
## <center> Gradient boosting

Your task is to beat at least 2 benchmarks in this [Kaggle Inclass competition](https://www.kaggle.com/c/flight-delays-spring-2018). Here you won’t be provided with detailed instructions. We only give you a brief description of how the second benchmark was achieved using Xgboost. Hopefully, at this stage of the course, it's enough for you to take a quick look at the data in order to understand that this is the type of task where gradient boosting will perform well. Most likely it will be Xgboost, however, we’ve got plenty of categorical features here.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [5]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [6]:
import catboost as cb

In [7]:
train = pd.read_csv('../../data/flight_delays_train.csv.zip')
test = pd.read_csv('../../data/flight_delays_test.csv.zip')

In [8]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [9]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Given flight departure time, carrier's code, departure airport, destination location, and flight distance, you have to predict departure delay for more than 15 minutes. 

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
Month                100000 non-null object
DayofMonth           100000 non-null object
DayOfWeek            100000 non-null object
DepTime              100000 non-null int64
UniqueCarrier        100000 non-null object
Origin               100000 non-null object
Dest                 100000 non-null object
Distance             100000 non-null int64
dep_delayed_15min    100000 non-null object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [11]:
import math

def prepData(df):
#     for col in ['Month', 'DayofMonth', 'DayOfWeek']:
#         df[col] = df[col].str[2:].astype(int)
    df['Flight'] = df['Origin'].map(str) + '-' + df['Dest'].map(str)
    df['DepTimeHour'] = df['DepTime'].apply(lambda x: math.floor(x/100)%24)
    df['DepTimeMin'] = df['DepTime'].apply(lambda x: x%100)
    #df.drop(['DepTime'], axis=1,inplace = True)
    return df

In [12]:
train = prepData(train)
test = prepData(test)
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Flight,DepTimeHour,DepTimeMin
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,ATL-DFW,19,34
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,PIT-MCO,15,48
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,RDU-CLE,14,22
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,DEN-MEM,10,15
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW-OMA,18,28


In [13]:
avoid_cols = ['dep_delayed_15min']
cols = [col for col in train.columns if col not in avoid_cols]
cat_cols = ['Month','DayofMonth', 'DayOfWeek',
            'UniqueCarrier','Origin','Dest','Flight', 'DepTimeHour', 'DepTimeMin']#,'DepTimeHour'
print(cols)
print(cat_cols)

['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 'Flight', 'DepTimeHour', 'DepTimeMin']
['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'Flight', 'DepTimeHour', 'DepTimeMin']


In [14]:
X_train = train[cols]#prepData(train).drop(['dep_delayed_15min'], axis=1)
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[cols]#prepData(test)

# X_train = train[['Distance', 'DepTime']].values
# y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
# X_test = test[['Distance', 'DepTime']].values


In [15]:
def cat_df(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'object' or col_type.name == 'category':
            df[col] = df[col].astype('category').cat.codes + 1

In [16]:
cat_df(X_train)
cat_df(X_test)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
Month            100000 non-null int8
DayofMonth       100000 non-null int8
DayOfWeek        100000 non-null int8
DepTime          100000 non-null int64
UniqueCarrier    100000 non-null int8
Origin           100000 non-null int16
Dest             100000 non-null int16
Distance         100000 non-null int64
Flight           100000 non-null int16
DepTimeHour      100000 non-null int64
DepTimeMin       100000 non-null int64
dtypes: int16(3), int64(4), int8(4)
memory usage: 4.0 MB


In [17]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
Month            100000 non-null int8
DayofMonth       100000 non-null int8
DayOfWeek        100000 non-null int8
DepTime          100000 non-null int64
UniqueCarrier    100000 non-null int8
Origin           100000 non-null int16
Dest             100000 non-null int16
Distance         100000 non-null int64
Flight           100000 non-null int16
DepTimeHour      100000 non-null int64
DepTimeMin       100000 non-null int64
dtypes: int16(3), int64(4), int8(4)
memory usage: 4.0 MB


In [18]:
X_train_part, X_valid, y_train_part, y_valid = \
    train_test_split(X_train, y_train, 
                     test_size=0.3, random_state=17)

In [19]:
X_train_part.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Flight,DepTimeHour,DepTimeMin
98187,7,10,1,1148,14,21,176,559,304,11,48
25139,11,5,7,645,13,229,36,612,3618,6,45
83491,7,27,3,525,11,125,160,102,1772,5,25
8295,4,25,7,1336,7,205,253,1249,3253,13,36
65284,11,31,2,1653,1,80,235,1171,1331,16,53


We'll train lgboost with default parameters on part of data and estimate holdout ROC AUC.

In [28]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = CatBoostClassifier()#cb.CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(X_train_part, y_train_part)

0:	learn: 0.6775578	total: 156ms	remaining: 46.7s
1:	learn: 0.6621038	total: 236ms	remaining: 35.2s
2:	learn: 0.6477219	total: 306ms	remaining: 30.3s
3:	learn: 0.6350903	total: 378ms	remaining: 28s
4:	learn: 0.6224066	total: 445ms	remaining: 26.2s
5:	learn: 0.6119201	total: 515ms	remaining: 25.2s
6:	learn: 0.6012146	total: 577ms	remaining: 24.2s
7:	learn: 0.5910735	total: 648ms	remaining: 23.7s
8:	learn: 0.5823533	total: 715ms	remaining: 23.1s
9:	learn: 0.5736657	total: 784ms	remaining: 22.7s
10:	learn: 0.5656976	total: 839ms	remaining: 22s
11:	learn: 0.5589009	total: 902ms	remaining: 21.7s
12:	learn: 0.5516925	total: 961ms	remaining: 21.2s
13:	learn: 0.5450964	total: 1.01s	remaining: 20.7s
14:	learn: 0.5386720	total: 1.07s	remaining: 20.4s
15:	learn: 0.5333436	total: 1.13s	remaining: 20.1s
16:	learn: 0.5280534	total: 1.19s	remaining: 19.8s
17:	learn: 0.5229571	total: 1.25s	remaining: 19.6s
18:	learn: 0.5178478	total: 1.31s	remaining: 19.4s
19:	learn: 0.5136387	total: 1.38s	remaining: 

163:	learn: 0.4401815	total: 9.95s	remaining: 8.25s
164:	learn: 0.4401446	total: 10s	remaining: 8.2s
165:	learn: 0.4400923	total: 10.1s	remaining: 8.14s
166:	learn: 0.4400598	total: 10.1s	remaining: 8.08s
167:	learn: 0.4400296	total: 10.2s	remaining: 8.02s
168:	learn: 0.4400012	total: 10.3s	remaining: 7.95s
169:	learn: 0.4399530	total: 10.3s	remaining: 7.9s
170:	learn: 0.4398939	total: 10.4s	remaining: 7.83s
171:	learn: 0.4398201	total: 10.4s	remaining: 7.78s
172:	learn: 0.4397787	total: 10.5s	remaining: 7.71s
173:	learn: 0.4397257	total: 10.6s	remaining: 7.65s
174:	learn: 0.4396973	total: 10.6s	remaining: 7.6s
175:	learn: 0.4396543	total: 10.7s	remaining: 7.54s
176:	learn: 0.4396157	total: 10.8s	remaining: 7.48s
177:	learn: 0.4395878	total: 10.8s	remaining: 7.41s
178:	learn: 0.4395538	total: 10.9s	remaining: 7.35s
179:	learn: 0.4395265	total: 10.9s	remaining: 7.29s
180:	learn: 0.4394613	total: 11s	remaining: 7.23s
181:	learn: 0.4394072	total: 11.1s	remaining: 7.17s
182:	learn: 0.43936

26:	learn: 0.4924507	total: 1.38s	remaining: 14s
27:	learn: 0.4900681	total: 1.43s	remaining: 13.9s
28:	learn: 0.4877476	total: 1.48s	remaining: 13.8s
29:	learn: 0.4858019	total: 1.52s	remaining: 13.6s
30:	learn: 0.4834514	total: 1.56s	remaining: 13.5s
31:	learn: 0.4813110	total: 1.6s	remaining: 13.4s
32:	learn: 0.4791222	total: 1.65s	remaining: 13.3s
33:	learn: 0.4773232	total: 1.69s	remaining: 13.2s
34:	learn: 0.4757725	total: 1.74s	remaining: 13.1s
35:	learn: 0.4744012	total: 1.78s	remaining: 13.1s
36:	learn: 0.4729401	total: 1.83s	remaining: 13s
37:	learn: 0.4714102	total: 1.89s	remaining: 13s
38:	learn: 0.4702260	total: 1.96s	remaining: 13.1s
39:	learn: 0.4692368	total: 2.02s	remaining: 13.1s
40:	learn: 0.4682683	total: 2.08s	remaining: 13.2s
41:	learn: 0.4673098	total: 2.14s	remaining: 13.1s
42:	learn: 0.4662233	total: 2.2s	remaining: 13.2s
43:	learn: 0.4652866	total: 2.26s	remaining: 13.1s
44:	learn: 0.4643032	total: 2.31s	remaining: 13.1s
45:	learn: 0.4635333	total: 2.37s	remai

186:	learn: 0.4408447	total: 11.2s	remaining: 6.78s
187:	learn: 0.4408163	total: 11.3s	remaining: 6.73s
188:	learn: 0.4407678	total: 11.4s	remaining: 6.68s
189:	learn: 0.4407444	total: 11.5s	remaining: 6.63s
190:	learn: 0.4407200	total: 11.5s	remaining: 6.57s
191:	learn: 0.4406909	total: 11.6s	remaining: 6.5s
192:	learn: 0.4406661	total: 11.6s	remaining: 6.44s
193:	learn: 0.4406384	total: 11.7s	remaining: 6.38s
194:	learn: 0.4405873	total: 11.7s	remaining: 6.31s
195:	learn: 0.4405548	total: 11.8s	remaining: 6.25s
196:	learn: 0.4405349	total: 11.8s	remaining: 6.19s
197:	learn: 0.4404800	total: 11.9s	remaining: 6.13s
198:	learn: 0.4404564	total: 12s	remaining: 6.07s
199:	learn: 0.4404285	total: 12s	remaining: 6s
200:	learn: 0.4403988	total: 12.1s	remaining: 5.95s
201:	learn: 0.4403629	total: 12.2s	remaining: 5.9s
202:	learn: 0.4403429	total: 12.2s	remaining: 5.84s
203:	learn: 0.4403022	total: 12.3s	remaining: 5.79s
204:	learn: 0.4402586	total: 12.4s	remaining: 5.74s
205:	learn: 0.4402151

49:	learn: 0.4625929	total: 3.55s	remaining: 17.8s
50:	learn: 0.4617627	total: 3.61s	remaining: 17.6s
51:	learn: 0.4611629	total: 3.67s	remaining: 17.5s
52:	learn: 0.4604426	total: 3.74s	remaining: 17.4s
53:	learn: 0.4599833	total: 3.81s	remaining: 17.4s
54:	learn: 0.4594758	total: 3.88s	remaining: 17.3s
55:	learn: 0.4588552	total: 3.96s	remaining: 17.3s
56:	learn: 0.4584137	total: 4.03s	remaining: 17.2s
57:	learn: 0.4580250	total: 4.08s	remaining: 17s
58:	learn: 0.4574872	total: 4.14s	remaining: 16.9s
59:	learn: 0.4570928	total: 4.19s	remaining: 16.8s
60:	learn: 0.4565765	total: 4.25s	remaining: 16.7s
61:	learn: 0.4561026	total: 4.32s	remaining: 16.6s
62:	learn: 0.4556748	total: 4.39s	remaining: 16.5s
63:	learn: 0.4552651	total: 4.47s	remaining: 16.5s
64:	learn: 0.4550432	total: 4.52s	remaining: 16.3s
65:	learn: 0.4547729	total: 4.59s	remaining: 16.3s
66:	learn: 0.4544277	total: 4.65s	remaining: 16.2s
67:	learn: 0.4541759	total: 4.69s	remaining: 16s
68:	learn: 0.4538256	total: 4.72s	r

...

17:	learn: 0.4401309	total: 2.71s	remaining: 42.4s
18:	learn: 0.4390986	total: 2.89s	remaining: 42.7s
19:	learn: 0.4385125	total: 2.98s	remaining: 41.7s
20:	learn: 0.4380139	total: 3.06s	remaining: 40.7s
21:	learn: 0.4372824	total: 3.24s	remaining: 41s
22:	learn: 0.4366157	total: 3.39s	remaining: 40.9s
23:	learn: 0.4360595	total: 3.56s	remaining: 40.9s
24:	learn: 0.4353310	total: 3.7s	remaining: 40.7s
25:	learn: 0.4349413	total: 3.83s	remaining: 40.4s
26:	learn: 0.4345809	total: 3.92s	remaining: 39.6s
27:	learn: 0.4342215	total: 4.09s	remaining: 39.8s
28:	learn: 0.4339140	total: 4.25s	remaining: 39.7s
29:	learn: 0.4333526	total: 4.42s	remaining: 39.8s
30:	learn: 0.4328252	total: 4.59s	remaining: 39.8s
31:	learn: 0.4326853	total: 4.66s	remaining: 39s
32:	learn: 0.4326457	total: 4.73s	remaining: 38.3s
33:	learn: 0.4322682	total: 4.89s	remaining: 38.3s
34:	learn: 0.4315889	total: 5.04s	remaining: 38.2s
35:	learn: 0.4309553	total: 5.2s	remaining: 38.2s
36:	learn: 0.4307594	total: 5.28s	rem

178:	learn: 0.4038359	total: 24s	remaining: 16.2s
179:	learn: 0.4035079	total: 24.1s	remaining: 16.1s
180:	learn: 0.4033649	total: 24.2s	remaining: 15.9s
181:	learn: 0.4030413	total: 24.4s	remaining: 15.8s
182:	learn: 0.4028780	total: 24.6s	remaining: 15.7s
183:	learn: 0.4026878	total: 24.7s	remaining: 15.6s
184:	learn: 0.4024852	total: 24.9s	remaining: 15.5s
185:	learn: 0.4024183	total: 25s	remaining: 15.4s
186:	learn: 0.4023866	total: 25.2s	remaining: 15.2s
187:	learn: 0.4023633	total: 25.3s	remaining: 15.1s
188:	learn: 0.4021152	total: 25.5s	remaining: 15s
189:	learn: 0.4021143	total: 25.6s	remaining: 14.8s
190:	learn: 0.4021060	total: 25.7s	remaining: 14.7s
191:	learn: 0.4020494	total: 25.8s	remaining: 14.5s
192:	learn: 0.4020114	total: 26s	remaining: 14.4s
193:	learn: 0.4020033	total: 26.1s	remaining: 14.2s
194:	learn: 0.4020033	total: 26.1s	remaining: 14.1s
195:	learn: 0.4019685	total: 26.2s	remaining: 13.9s
196:	learn: 0.4017915	total: 26.4s	remaining: 13.8s
197:	learn: 0.401656

40:	learn: 0.4234346	total: 3.83s	remaining: 24.2s
41:	learn: 0.4227543	total: 3.93s	remaining: 24.2s
42:	learn: 0.4218326	total: 4.03s	remaining: 24.1s
43:	learn: 0.4212913	total: 4.13s	remaining: 24s
44:	learn: 0.4207596	total: 4.22s	remaining: 23.9s
45:	learn: 0.4201807	total: 4.32s	remaining: 23.8s
46:	learn: 0.4193500	total: 4.42s	remaining: 23.8s
47:	learn: 0.4188354	total: 4.51s	remaining: 23.7s
48:	learn: 0.4184184	total: 4.6s	remaining: 23.6s
49:	learn: 0.4178601	total: 4.69s	remaining: 23.5s
50:	learn: 0.4172494	total: 4.78s	remaining: 23.4s
51:	learn: 0.4163475	total: 4.88s	remaining: 23.3s
52:	learn: 0.4157134	total: 4.98s	remaining: 23.2s
53:	learn: 0.4151348	total: 5.08s	remaining: 23.2s
54:	learn: 0.4145596	total: 5.17s	remaining: 23.1s
55:	learn: 0.4142401	total: 5.26s	remaining: 22.9s
56:	learn: 0.4138779	total: 5.35s	remaining: 22.8s
57:	learn: 0.4134169	total: 5.44s	remaining: 22.7s
58:	learn: 0.4129137	total: 5.53s	remaining: 22.6s
59:	learn: 0.4122896	total: 5.63s	

202:	learn: 0.3498446	total: 19.3s	remaining: 9.2s
203:	learn: 0.3495466	total: 19.3s	remaining: 9.1s
204:	learn: 0.3491045	total: 19.4s	remaining: 9.01s
205:	learn: 0.3488790	total: 19.5s	remaining: 8.91s
206:	learn: 0.3486228	total: 19.6s	remaining: 8.81s
207:	learn: 0.3478957	total: 19.7s	remaining: 8.72s
208:	learn: 0.3473733	total: 19.8s	remaining: 8.63s
209:	learn: 0.3471156	total: 19.9s	remaining: 8.53s
210:	learn: 0.3468798	total: 20s	remaining: 8.44s
211:	learn: 0.3465917	total: 20.1s	remaining: 8.34s
212:	learn: 0.3461357	total: 20.2s	remaining: 8.24s
213:	learn: 0.3458307	total: 20.3s	remaining: 8.15s
214:	learn: 0.3453576	total: 20.4s	remaining: 8.05s
215:	learn: 0.3450836	total: 20.5s	remaining: 7.96s
216:	learn: 0.3448469	total: 20.6s	remaining: 7.87s
217:	learn: 0.3445741	total: 20.7s	remaining: 7.77s
218:	learn: 0.3441713	total: 20.8s	remaining: 7.68s
219:	learn: 0.3437685	total: 20.8s	remaining: 7.58s
220:	learn: 0.3434535	total: 20.9s	remaining: 7.49s
221:	learn: 0.34

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=<catboost.core.CatBoostClassifier object at 0x7f4cb4283a58>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'depth': [4, 7, 10], 'learning_rate': [0.03, 0.1, 0.15], 'l2_leaf_reg': [1, 4, 9], 'iterations': [300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [29]:
# cb_model.best_estimator_
cb_model.best_score_

0.7308997411372816

In [30]:
cb_model.best_params_

{'depth': 10, 'iterations': 300, 'l2_leaf_reg': 9, 'learning_rate': 0.1}

In [21]:
from catboost import CatBoostClassifier

cat_features_index = [0,1,2,4,5,6,8,9]

cb_model = CatBoostClassifier(eval_metric="AUC", one_hot_max_size = 31,
                              depth=10, iterations= 300, l2_leaf_reg= 9, learning_rate= 0.1)
#50,num_leaves=900, ,objective='binary',metric = 'auc',
cb_model.fit(X_train_part, y_train_part, cat_features =cat_features_index
              )#,feature_name = cols
cb_valid_pred = cb_model.predict_proba(X_valid)[:,1]

roc_auc_score(y_valid, cb_valid_pred)

0:	total: 230ms	remaining: 1m 8s
1:	total: 367ms	remaining: 54.6s
2:	total: 530ms	remaining: 52.5s
3:	total: 664ms	remaining: 49.2s
4:	total: 813ms	remaining: 48s
5:	total: 952ms	remaining: 46.7s
6:	total: 1.1s	remaining: 46s
7:	total: 1.24s	remaining: 45.4s
8:	total: 1.4s	remaining: 45.3s
9:	total: 1.52s	remaining: 44.2s
10:	total: 1.66s	remaining: 43.5s
11:	total: 1.81s	remaining: 43.3s
12:	total: 1.96s	remaining: 43.4s
13:	total: 2.1s	remaining: 42.8s
14:	total: 2.24s	remaining: 42.6s
15:	total: 2.37s	remaining: 42.1s
16:	total: 2.52s	remaining: 42s
17:	total: 2.66s	remaining: 41.6s
18:	total: 2.8s	remaining: 41.4s
19:	total: 2.94s	remaining: 41.1s
20:	total: 2.97s	remaining: 39.5s
21:	total: 3.14s	remaining: 39.7s
22:	total: 3.32s	remaining: 40s
23:	total: 3.5s	remaining: 40.3s
24:	total: 3.65s	remaining: 40.2s
25:	total: 3.81s	remaining: 40.2s
26:	total: 3.97s	remaining: 40.1s
27:	total: 4.13s	remaining: 40.1s
28:	total: 4.29s	remaining: 40.1s
29:	total: 4.45s	remaining: 40s
30:	t

240:	total: 39.4s	remaining: 9.64s
241:	total: 39.5s	remaining: 9.47s
242:	total: 39.7s	remaining: 9.3s
243:	total: 39.8s	remaining: 9.14s
244:	total: 40s	remaining: 8.97s
245:	total: 40.1s	remaining: 8.8s
246:	total: 40.3s	remaining: 8.64s
247:	total: 40.4s	remaining: 8.47s
248:	total: 40.5s	remaining: 8.3s
249:	total: 40.7s	remaining: 8.14s
250:	total: 40.9s	remaining: 7.97s
251:	total: 41s	remaining: 7.81s
252:	total: 41.1s	remaining: 7.64s
253:	total: 41.3s	remaining: 7.47s
254:	total: 41.4s	remaining: 7.31s
255:	total: 41.6s	remaining: 7.15s
256:	total: 41.7s	remaining: 6.98s
257:	total: 41.9s	remaining: 6.81s
258:	total: 42s	remaining: 6.65s
259:	total: 42.1s	remaining: 6.48s
260:	total: 42.3s	remaining: 6.32s
261:	total: 42.4s	remaining: 6.15s
262:	total: 42.6s	remaining: 5.99s
263:	total: 42.7s	remaining: 5.83s
264:	total: 42.9s	remaining: 5.66s
265:	total: 43s	remaining: 5.5s
266:	total: 43.2s	remaining: 5.34s
267:	total: 43.3s	remaining: 5.17s
268:	total: 43.5s	remaining: 5.0

0.7380032210733127

Now we do the same with the whole training set, make predictions to test set and form a submission file. This is how you beat the first benchmark. 

In [23]:
cb_model.fit(X_train, y_train, cat_features =cat_features_index)
cb_test_pred = cb_model.predict_proba(X_test)[:, 1]

pd.Series(cb_test_pred, 
          name='dep_delayed_15min').to_csv('cb_fe.csv', 
                                           index_label='id', header=True)

0:	total: 174ms	remaining: 52.1s
1:	total: 251ms	remaining: 37.4s
2:	total: 435ms	remaining: 43.1s
3:	total: 587ms	remaining: 43.4s
4:	total: 768ms	remaining: 45.3s
5:	total: 820ms	remaining: 40.2s
6:	total: 871ms	remaining: 36.4s
7:	total: 1.05s	remaining: 38.4s
8:	total: 1.2s	remaining: 38.9s
9:	total: 1.38s	remaining: 40s
10:	total: 1.54s	remaining: 40.5s
11:	total: 1.7s	remaining: 40.8s
12:	total: 1.87s	remaining: 41.4s
13:	total: 2.04s	remaining: 41.7s
14:	total: 2.21s	remaining: 41.9s
15:	total: 2.37s	remaining: 42.1s
16:	total: 2.53s	remaining: 42.2s
17:	total: 2.71s	remaining: 42.4s
18:	total: 2.86s	remaining: 42.3s
19:	total: 3.04s	remaining: 42.5s
20:	total: 3.22s	remaining: 42.7s
21:	total: 3.4s	remaining: 43s
22:	total: 3.58s	remaining: 43.1s
23:	total: 3.76s	remaining: 43.3s
24:	total: 3.93s	remaining: 43.2s
25:	total: 4.1s	remaining: 43.2s
26:	total: 4.27s	remaining: 43.2s
27:	total: 4.47s	remaining: 43.4s
28:	total: 4.64s	remaining: 43.4s
29:	total: 4.83s	remaining: 43.4

242:	total: 46.2s	remaining: 10.8s
243:	total: 46.3s	remaining: 10.6s
244:	total: 46.5s	remaining: 10.4s
245:	total: 46.7s	remaining: 10.3s
246:	total: 46.9s	remaining: 10.1s
247:	total: 47.1s	remaining: 9.87s
248:	total: 47.3s	remaining: 9.68s
249:	total: 47.6s	remaining: 9.52s
250:	total: 47.8s	remaining: 9.33s
251:	total: 48s	remaining: 9.14s
252:	total: 48.2s	remaining: 8.96s
253:	total: 48.4s	remaining: 8.77s
254:	total: 48.7s	remaining: 8.59s
255:	total: 48.9s	remaining: 8.4s
256:	total: 49s	remaining: 8.21s
257:	total: 49.4s	remaining: 8.04s
258:	total: 49.7s	remaining: 7.86s
259:	total: 50s	remaining: 7.69s
260:	total: 50.3s	remaining: 7.52s
261:	total: 50.7s	remaining: 7.36s
262:	total: 51s	remaining: 7.17s
263:	total: 51.2s	remaining: 6.98s
264:	total: 51.4s	remaining: 6.78s
265:	total: 51.6s	remaining: 6.59s
266:	total: 51.7s	remaining: 6.39s
267:	total: 51.9s	remaining: 6.2s
268:	total: 52.1s	remaining: 6s
269:	total: 52.3s	remaining: 5.81s
270:	total: 52.4s	remaining: 5.61

The second benchmark in the leaderboard was achieved as follows:

- Features `Distance` and `DepTime` were taken unchanged
- A feature `Flight` was created from features `Origin` and `Dest`
- Features `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` and `Flight` were transformed with OHE (`LabelBinarizer`)
- Logistic regression and gradient boosting (xgboost) were trained. Xgboost hyperparameters were tuned via cross-validation. First, the hyperparameters responsible for model complexity were optimized, then the number of trees was fixed at 500 and learning step was tuned.
- Predicted probabilities were made via cross-validation using `cross_val_predict`. A linear mixture of logistic regression and gradient boosting predictions was set in the form $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, where $p_{logit}$ is a probability of class 1, predicted by logistic regression, and $p_{xgb}$ – the same for xgboost. $w_1$ weight was selected manually.
- A similar combination of predictions was made for test set. 

Following the same steps is not mandatory. That’s just a description of how the result was achieved by the author of this assignment. Perhaps you might not want to follow the same steps, and instead, let’s say, add a couple of good features and train a random forest of a thousand trees.

Good luck!